In [1]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential, load_model  # Import load_model here
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from PIL import Image
import cv2
import datetime

In [2]:
# Build the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [3]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)


In [4]:
# Load training and test data
train_set = train_datagen.flow_from_directory(
        "D:\\Semester 8\\COMPUTER VISION AND PATTERN RECOGNITION [A]\\Assignment Final\\FaceMaskDetector-master\\data\\train",
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')

test_set = test_datagen.flow_from_directory(
        "D:\\Semester 8\\COMPUTER VISION AND PATTERN RECOGNITION [A]\\Assignment Final\\FaceMaskDetector-master\\data\\test",
        target_size=(150, 150),
        batch_size=16,
        class_mode='binary')


Found 1314 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [5]:
# Training the model
model.fit(
    train_set,
    epochs=10,
    validation_data=test_set,
    callbacks=[ModelCheckpoint('mymodel.h5', save_best_only=True)]
)

Epoch 1/10
83/83 [==============================] - 45s 529ms/step - loss: 0.3678 - accuracy: 0.8120 - val_loss: 0.1333 - val_accuracy: 0.9433
Epoch 2/10
83/83 [==============================] - 38s 462ms/step - loss: 0.1509 - accuracy: 0.9460 - val_loss: 0.1060 - val_accuracy: 0.9536
Epoch 3/10
83/83 [==============================] - 35s 412ms/step - loss: 0.1158 - accuracy: 0.9619 - val_loss: 0.0978 - val_accuracy: 0.9588
Epoch 4/10
83/83 [==============================] - 52s 628ms/step - loss: 0.1010 - accuracy: 0.9658 - val_loss: 0.1721 - val_accuracy: 0.9330
Epoch 5/10
83/83 [==============================] - 36s 437ms/step - loss: 0.1414 - accuracy: 0.9444 - val_loss: 0.0964 - val_accuracy: 0.9536
Epoch 6/10
83/83 [==============================] - 37s 440ms/step - loss: 0.1503 - accuracy: 0.9498 - val_loss: 0.0935 - val_accuracy: 0.9691
Epoch 7/10
83/83 [==============================] - 32s 382ms/step - loss: 0.0774 - accuracy: 0.9718 - val_loss: 0.0865 - val_accuracy: 0.9639

In [6]:
# Load the trained model
mymodel = load_model('mymodel.h5')

In [7]:
# Evaluate the model on the test data
loss, accuracy = mymodel.evaluate(test_set)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

test_image_path = r"D:\Semester 8\COMPUTER VISION AND PATTERN RECOGNITION [A]\Assignment Final\FaceMaskDetector-master\data\test\with_mask\1-with-mask.jpg"
test_image = Image.open(test_image_path)
test_image = test_image.resize((150, 150))
test_image_array = np.array(test_image, dtype=np.float32)  # Convert to float32
test_image_array = np.expand_dims(test_image_array, axis=0)
test_image_array /= 255.0  # Normalize pixel values

prediction = mymodel.predict(test_image_array)[0][0]

if prediction <= 0.5:
    print("Prediction: With Mask")
else:
    print("Prediction: Without Mask")

13/13 [==============================] - 2s 119ms/step - loss: 0.0752 - accuracy: 0.9691
Test Loss: 0.07524131238460541
Test Accuracy: 0.969072163105011
1/1 [==============================] - 1s 562ms/step
Prediction: With Mask


In [8]:
# Evaluate the model on the test data
loss, accuracy = mymodel.evaluate(test_set)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

test_image_path = r"D:\Semester 8\COMPUTER VISION AND PATTERN RECOGNITION [A]\Assignment Final\FaceMaskDetector-master\data\test\without_mask\0.jpg"
test_image = Image.open(test_image_path)
test_image = test_image.resize((150, 150))
test_image_array = np.array(test_image, dtype=np.float32)  # Convert to float32
test_image_array = np.expand_dims(test_image_array, axis=0)
test_image_array /= 255.0  # Normalize pixel values

prediction = mymodel.predict(test_image_array)[0][0]

if prediction <= 0.5:
    print("Prediction: With Mask")
else:
    print("Prediction: Without Mask")

13/13 [==============================] - 2s 117ms/step - loss: 0.0752 - accuracy: 0.9691
Test Loss: 0.07524131238460541
Test Accuracy: 0.969072163105011
1/1 [==============================] - 0s 73ms/step
Prediction: Without Mask


In [17]:
import cv2
import numpy as np
from keras.models import load_model

# Load the trained model
mymodel = load_model('mymodel.h5')

# Start capturing video from the laptop's camera
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    
    # Check if the frame was captured successfully
    if not ret:
        print("Failed to capture frame from camera.")
        break
    
    # Resize the frame to match the input shape of the model
    resized_frame = cv2.resize(frame, (150, 150))
    
    # Preprocess the frame for prediction
    input_image = np.expand_dims(resized_frame, axis=0)
    input_image = input_image / 255.0  # Normalize pixel values
    
    # Predict using the model
    prediction = mymodel.predict(input_image)[0][0]
    
    # Display the prediction result
    label = "With Mask" if prediction >= 0.5 else "Without Mask"
    color = (0, 255, 0) if prediction >= 0.5 else (0, 0, 255)
    cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    
    # Display the frame
    cv2.imshow('Face Mask Detection', frame)
    
    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 70ms/step
